In [1]:
import networkx as nx
import matplotlib.pyplot as plt
from typing import Type
from simulator import CovertGenerator, DarkGenerator
import numpy as np
import pandas as pd

/usr/local/lib/python3.8/site-packages/scipy/__init__.py:143: UserWarning: A NumPy version >=1.19.5 and <1.27.0 is required for this version of SciPy (detected version 1.17.3)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [9]:
import json

## Generator & Reader

In [16]:
def ba_generator(min_n: int=20, max_n: int=30, m: int=2) -> Type[nx.classes.graph.Graph]:
    n = np.random.randint(max_n - min_n + 1) + min_n
    return nx.barabasi_albert_graph(n ,m)

def covert_generator(min_n: int=20, max_n: int=30, density: int=0.1):
    generator = CovertGenerator(min_n, max_n, density)
    generator.simulate()
    return generator.G

def dark_generator(min_n: int=20, max_n: int=30, density: int=0.1):
    generator = DarkGenerator(min_n, max_n, density)
    generator.simulate()
    return generator.G

def write_gml(G: Type[nx.classes.graph.Graph], data_dir: str, file_name: str) -> None:
    nx.write_gml(G, data_dir+file_name)
    
def read_gml(data_dir: str, file_name: str) -> Type[nx.classes.graph.Graph]:
    G = nx.read_gml(data_dir + file_name)
    return nx.relabel_nodes(G, lambda x: int(x))

def get_real_graph(data_dir: str, file_name: str) -> Type[nx.classes.graph.Graph]:
    return read_gml(data_dir, file_name)

def getRobustness(g: Type[nx.classes.graph.Graph], sol: int) -> float:
    G = g.copy()
    GCCsize = len(max(nx.connected_components(G), key=len))
    G.remove_node(sol)
    newGCCsize = len(max(nx.connected_components(G), key=len))

    return (GCCsize - newGCCsize) / ((G.number_of_nodes() * G.number_of_nodes()))

In [18]:
def HXA(g: Type[nx.classes.graph.Graph], method: str) -> (list, list):
    # 'HDA', 'HBA', 'HCA', 'HPRA'
    sol = []
    reward = []
    G = g.copy()
    while (nx.number_of_edges(G)>0):
        if method == 'HDA':
            dc = nx.degree_centrality(G)
        elif method == 'HBA':
            dc = nx.betweenness_centrality(G)
        elif method == 'HCA':
            dc = nx.closeness_centrality(G)
        elif method == 'HPRA':
            dc = nx.pagerank(G)
        keys = list(dc.keys())
        values = list(dc.values())
        maxTag = np.argmax(values)
        node = keys[maxTag]
        
        reward.append(getRobustness(G, int(node)))
        
        sol.append(int(node))
        G.remove_node(node)
        
    return sol, reward

## Covert & Dark 

In [7]:
iters = 100
for i in range(iters):
    G = dark_generator(min_n=200, max_n=200, density=0.01)
    write_gml(G, data_dir="./dark/", file_name=f"g_{i}")

In [23]:
# iters = 100
# for i in range(iters): 
#     G = covert_generator(min_n=200, max_n=200, density=0.01)
#     write_gml(G, data_dir="./covert/", file_name=f"g_{i}")

## Heuristic based

In [27]:
for i in range(iters):
    hxa_dct = dict()
    reward_dct = dict()
    g = read_gml(data_dir="./dark/", file_name=f"g_{i}")

    for method in ['HDA', 'HBA', 'HCA', 'HPRA']:
        sol, reward = HXA(g=g, method=method)
        hxa_dct[method] = sol
        reward_dct[method] = np.cumsum(reward).tolist()

    with open(f"./dark/node_hist/g_{i}.json", "w") as json_file:
        json.dump(hxa_dct, json_file) 
    with open(f"./dark/reward_hist/g_{i}.json", "w") as json_file:
        json.dump(reward_dct, json_file) 